# Ear Verification

In [8]:
from numpy import linalg
from menpo import io as mio
from menpo.visualize import print_dynamic
from sklearn.utils.fixes import bincount
import itertools
import warnings
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from dAAMs.lda import lda, predict, chunk, n_fold_generate
from menpo.feature import igo, hog, no_op, double_igo as digo, dsift, fast_dsift, hellinger_vector_128_dsift, gradient
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc
from menpo.model import PCAModel, PCAVectorModel
from dAAMs.fisher import FisherVector
import functools

data_cache = {}

%matplotlib inline

#### feature extractions

In [9]:
def mod_igos(pixels, double_angles=False):

    if len(pixels.shape) != 3:
        raise ValueError('IGOs only work on 2D images. Expects image data to be 3D, channels + shape.')
                
    n_img_chnls = pixels.shape[0]

    feat_chnls = 2
    if double_angles:
        feat_chnls = 4

    grad = gradient(pixels)
    grad_orient = np.angle(grad[:n_img_chnls] + 1j * grad[n_img_chnls:])

    if double_angles:
        grad_orient = np.hstack([grad_orient, 2*grad_orient])

    return grad_orient


#### classification functions

In [10]:
from dAAMs.lda import lda, predict, chunk, n_fold_generate, decision_function
class LDAWrap:
    def __init__(self):
        self.intercept_ = None
        self.coef_ = None
        self.classes_ = None
    def fit(self,train_X, train_y):
        self.intercept_, self.coef_, self.classes_ = lda(train_X, train_y)
    def predict(self, test_X):
        return predict(test_X, self.intercept_, self.coef_, self.classes_)
    def decision_function(self, test_X):
        return decision_function(test_X,self.intercept_, self.coef_)

In [11]:
class JBC:
    def __init__(self):
        self._A = None
        self._G = None
    def fit(self,train_X, train_y):
        X_I = []
        X_E = []
        for (v1,v2),label in zip(train_X, train_y):
            if label > 0:
                X_I.append(np.concatenate([v1,v2]).reshape(-1,1))
            else:
                X_E.append(np.concatenate([v1,v2]).reshape(-1,1))
                
        print('computing cov Matrixes with size: {} * {}'.format(len(X_I), len(X_I[0])))
        
        covI = np.cov(np.hstack(X_I))
        covE = np.cov(np.hstack(X_E))
        size = covI.shape[0] / 2
        inverseI = np.linalg.inv(covI)
        inverseE = np.linalg.inv(covE)
        
        
        _F = inverseI[0:size,0:size]
        _G = inverseI[0:size,size:]
        _temp = np.zeros(inverseE.shape)
        _temp[0:size,0:size] = _F
        _temp[size:,size:] = _F
        _A = (inverseE - _temp)[0:size,0:size]
        
        self._A = _A
        self._G = _G
        
    def predict(self, test_X):
        return self.decision_function(test_X) > 0
    
    def decision_function(self, test_X):
        def df(self, v1, v2):
            return v1.T.dot(self._A).dot(v1) + v2.T.dot(self._A).dot(v2) - 2 * v1.T.dot(self._G).dot(v2)
        
        return np.array([df(self,v1,v2) for v1,v2 in test_X])
        

#### preprocessing functions

In [12]:
def pair_preprocess_cosine(trains,tests):
    ret_train = []
    ret_test = []
    
    for v1,v2 in trains:
        phi1 = mod_igos(v1)
        phi2 = mod_igos(v2)
        dphi = phi1-phi2
        ret_train.append(np.mean(np.cos(dphi)))
        
    for v1,v2 in tests:
        phi1 = mod_igos(v1)
        phi2 = mod_igos(v2)
        dphi = phi1-phi2
        ret_test.append(np.mean(np.cos(dphi)))
        
    return ret_train, ret_test

In [13]:
def pair_preprocess_concat(trains,tests):
    ret_train = []
    ret_test = []
    
    for v1,v2 in trains:
        ret_train.append(np.concatenate([v1,v2]))
        
    for v1,v2 in tests:
        ret_test.append(np.concatenate([v1,v2]))
        
    return ret_train, ret_test

In [14]:
def pair_preprocess_diff(trains,tests):
    ret_train = []
    ret_test = []
    
    for v1,v2 in trains:
        ret_train.append(v1-v2)
        
    for v1,v2 in tests:
        ret_test.append(v1-v2)
        
    return ret_train, ret_test

In [15]:
def pair_preprocess_l2norm(trains,tests):
    ret_train = []
    ret_test = []
    
    for v1,v2 in trains:
        ret_train.append(np.linalg.norm(v1-v2))
        
    for v1,v2 in tests:
        ret_test.append(np.linalg.norm(v1-v2))
        
    return ret_train, ret_test

In [16]:
import numpy as np
from sklearn.decomposition import PCA
def pair_preprocess_pca(trains,tests, ratio=0.9):
    ret_train = []
    ret_test = []
    
    tr1,tr2 = list(zip(*trains))
    t1,t2 = list(zip(*tests))
      
    pca = PCAVectorModel(tr1 + tr2)
    
    if ratio < 1:
        n_active_component = np.argwhere(pca.eigenvalues_cumulative_ratio() > ratio)[0][0]
    else:
        n_active_component = ratio
    
    pca.n_active_components = n_active_component
    
    
    ret_train,ret_test = list(zip([pca.project(vec) for vec in tr1],[pca.project(vec) for vec in tr2])),list(zip([pca.project(vec) for vec in t1],[pca.project(vec) for vec in t2]))
    
    return ret_train, ret_test

In [17]:
def pair_preprocess_fv(trains,tests, K=128):
    ret_train = []
    ret_test = []
    
    tr1,tr2 = list(zip(*trains))
    t1,t2 = list(zip(*tests))
    
    fv = FisherVector(tr1+tr2, K=K)
    
    ret_train,ret_test = list(zip([fv.generate(vec, normalise=True) for vec in tr1],[fv.generate(vec, normalise=True) for vec in tr2])),list(zip([fv.generate(vec, normalise=True) for vec in t1],[fv.generate(vec, normalise=True) for vec in t2]))
    
    return ret_train, ret_test

In [18]:
def pair_preprocess_fv_all(trains,tests, K=128):
    ret_train = []
    ret_test = []
    
    tr1,tr2 = list(zip(*trains))
    t1,t2 = list(zip(*tests))
    
    fv = FisherVector(tr1+tr2, K=K)
    
    ret_train,ret_test = [fv.generate([v1,v2], normalise=True) for v1,v2 in trains],[fv.generate([v1,v2], normalise=True) for v1,v2 in tests]
    
    return ret_train, ret_test

In [19]:
def pair_preprocess_fv_l2norm(trains,tests, K=128):
    ret_train = []
    ret_test = []
    
    tr1,tr2 = list(zip(*trains))
    t1,t2 = list(zip(*tests))
    
    fv = FisherVector(tr1+tr2, K=K)
    
    ret_train,ret_test = [np.linalg.norm(fv.generate([v1,v2], normalise=True)) for v1,v2 in trains],[np.linalg.norm(fv.generate([v1,v2], normalise=True)) for v1,v2 in tests]
    
    return ret_train, ret_test

In [20]:
def pair_preprocess_pca_fv(trains,tests, ratio=0.9, K=128):
    ret_train, ret_test = pair_preprocess_pca(trains,tests, ratio)
    return pair_preprocess_fv(ret_train, ret_test, K=K)

In [21]:
def pair_preprocess_pca_fv_all(trains,tests, ratio=0.9, K=128):
    ret_train, ret_test = pair_preprocess_pca(trains,tests, ratio)
    return pair_preprocess_fv_all(ret_train, ret_test, K=K)

In [22]:
def pair_preprocess_pca_fv_all_l2norm(trains,tests, ratio=0.9, K=128):
    ret_train, ret_test = pair_preprocess_pca(trains,tests, ratio)
    return pair_preprocess_fv_l2norm(ret_train, ret_test, K=K)

In [23]:
def pair_preprocess_pca_fv_l2norm(trains,tests, ratio=0.9, K=128):
    ret_train, ret_test = pair_preprocess_pca(trains,tests, ratio)
    ret_train, ret_test = pair_preprocess_fv(ret_train, ret_test, K=K)
    return pair_preprocess_l2norm(ret_train, ret_test)

In [24]:
def pair_preprocess_pca_concat(trains,tests, ratio=0.9):
    ret_train, ret_test = pair_preprocess_pca(trains,tests, ratio)
    return pair_preprocess_concat(ret_train, ret_test)

In [25]:
def pair_preprocess_pca_l2norm(trains,tests, ratio=0.9):
    ret_train, ret_test = pair_preprocess_pca(trains,tests, ratio)
    return pair_preprocess_l2norm(ret_train, ret_test)

In [26]:
def pair_preprocess_pca_diff(trains,tests, ratio=0.9):
    ret_train, ret_test = pair_preprocess_pca(trains,tests, ratio)
    return pair_preprocess_diff(ret_train, ret_test)

In [27]:
def pair_preprocess_concat_pca(trains,tests, ratio=0.9):
    ret_train = []
    ret_test = []
    
    for v1,v2 in trains:
        ret_train.append(np.concatenate([v1,v2]))
        
    for v1,v2 in tests:
        ret_test.append(np.concatenate([v1,v2]))
        
    
    pca = PCAVectorModel(ret_train)
    pca.n_active_components = np.argwhere(pca.eigenvalues_cumulative_ratio() > ratio)[0][0]
    
    ret_train,ret_test = [pca.project(vec) for vec in ret_train],[pca.project(vec) for vec in ret_test]
        
    return ret_train, ret_test

### load model

In [28]:
def validation(folds, index, clf, postfix='', preprocess=None):
    train_X = []
    train_y = []
    test_X = []
    test_y = []
    
    for i,f in enumerate(folds):
        if not i == index:
            pairs,labels = list(zip(*f))
            train_X += list(pairs)
            train_y += labels
        else:
            pairs,labels = list(zip(*f))
            test_X += list(pairs)
            test_y += labels
            
            
    def query_data(fname):
        if not fname in data_cache.keys():
            img = mio.import_pickle(fname) if fname.suffix == '.pkl' else mio.import_image(fname).pixels.ravel()
            data_cache[fname] = img

        return fname

    for index, (fname1,fname2) in enumerate(train_X):
        train_X[index] = [data_cache[query_data(fname1)], data_cache[query_data(fname2)]]

    for index, (fname1,fname2) in enumerate(test_X):
        test_X[index] = [data_cache[query_data(fname1)], data_cache[query_data(fname2)]]
        
    if preprocess:
        train_X, test_X = preprocess(train_X, test_X)

    train_X = np.array(train_X).astype(float)
    test_X = np.array(test_X).astype(float)
    
    if len(train_X.shape) == 1:
        train_X = train_X.reshape((train_X.shape)+(1,))
    if len(test_X.shape) == 1:  
        test_X = test_X.reshape((test_X.shape)+(1,))
    
    train_y = np.array(train_y).astype(int)
    test_y = np.array(test_y).astype(int)
    
    
    
    clf.fit(train_X,train_y)

    py = clf.predict(test_X)
    
    
    acc = accuracy_score(test_y, py)
    
    try:
        fpr, tpr, _ = roc_curve(test_y, clf.decision_function(test_X), pos_label=1)
        roc_auc = auc(fpr, tpr)
    except Exception as e:
        print(e)
        fpr, tpr, roc_auc = [0,0,0]

    
    
    print(acc)
    return acc, fpr, tpr, roc_auc, clf, test_y,py

In [29]:
def intersect_validation(folds_A,folds_B, index, clf, postfix='', preprocess=None):
    intersect_A = [folds_B[index]] + [f for i,f in enumerate(folds_A) if not i == index ]
    intersect_B = [folds_A[index]] + [f for i,f in enumerate(folds_B) if not i == index ]
    return validation(intersect_A, 0, clf,postfix=postfix,preprocess=preprocess),validation(intersect_B, 0, clf,postfix=postfix,preprocess=preprocess)

In [30]:
def cross_intersect_validation(folds_A,folds_B, clf, postfix='', preprocess=None):
    results = []
    for i in range(len(folds_A)):
        results.append(intersect_validation(folds_A,folds_B, i, clf,postfix=postfix,preprocess=preprocess))
        
    return results

In [31]:
def cross_validation(folds, clf, postfix='', preprocess=None):
    results = []
    for i in range(len(folds)):
        results.append(validation(folds, i, clf,postfix=postfix,preprocess=preprocess))
        
    return results

In [32]:
def fold_norm(foldA, foldB):
    N_pairs_A = len(foldA[0])
    N_pairs_B = len(foldB[0])
    max_fold_pairs = np.min([N_pairs_A,N_pairs_B])
    shuffle_A = list(range(N_pairs_A))
    shuffle_B = list(range(N_pairs_B))
    np.random.shuffle(shuffle_A)
    np.random.shuffle(shuffle_B)
    shuffle_A = shuffle_A[:max_fold_pairs]
    shuffle_B = shuffle_B[:max_fold_pairs]
    
    return [list(np.array(foldA[0])[shuffle_A]),list(np.array(foldA[1])[shuffle_A])],[list(np.array(foldB[0])[shuffle_B]),list(np.array(foldB[1])[shuffle_B])]
    


In [33]:
def protocalAndPath(protocol, db_path, ext):
    db_path = Path(db_path)
    ret_protocol = []
    
    for fold in protocol:
        ret_fold = []
        for index, ((p1,p2), label) in enumerate(fold):
            ret_fold.append([[Path('{}/{}{}'.format(db_path,Path(p1).stem,ext)),Path('{}/{}{}'.format(db_path,Path(p2).stem,ext))],label])
        ret_protocol.append(ret_fold)

    return ret_protocol

## Experiments

In [55]:
db_path = Path('/homes/yz4009/wd/databases/ear/EarVerification/')
db_type = "bound"
# db_type = "aligned"

feature = '128dsift'
ext = '+{}.pkl'.format(feature) if not feature == '' else '.jpg'
preprocess_part = functools.partial(pair_preprocess_pca_fv, ratio=64, K=128)
preprocess = lambda x,y: pair_preprocess_pca(*preprocess_part(x,y), ratio=32)
# preprocess = pair_preprocess_l2norm
# preprocess = pair_preprocess_pca

In [56]:
clf = JBC()
# clf = svm.SVC(C=1)
# clf = LinearDiscriminantAnalysis()

#### VGGEAR

In [57]:
protocol_vggear = mio.import_pickle('{}/{}/protocol.pkl'.format(db_path,'VGGEAR'), encoding='latin1')

In [58]:
Bd = cross_validation(protocalAndPath(protocol_vggear, db_path/'VGGEAR'/db_type,ext), clf, postfix=feature, preprocess=preprocess)
print('Mean acc: {}'.format(np.mean(list(zip(*Bd))[0])))

computing cov Matrixes with size: 740 * 64
0.508108108108


/vol/atlas/homes/yz4009/miniconda/envs/gitdev/lib/python3.5/site-packages/ipykernel/__main__.py:23: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/vol/atlas/homes/yz4009/miniconda/envs/gitdev/lib/python3.5/site-packages/ipykernel/__main__.py:24: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/vol/atlas/homes/yz4009/miniconda/envs/gitdev/lib/python3.5/site-packages/ipykernel/__main__.py:26: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/vol/atlas/homes/yz4009/miniconda/envs/gitdev/lib/python3.5/site-packages/ipykernel/__main__.py:27: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/vol/atlas/homes/yz4009/miniconda/envs/gitdev/lib/python3.5/site-packages/ipykernel/__main__.py:28: VisibleDeprecationWarning: using a non-integ

computing cov Matrixes with size: 740 * 64
0.562162162162
computing cov Matrixes with size: 740 * 64
0.527027027027
computing cov Matrixes with size: 740 * 64
0.486486486486
computing cov Matrixes with size: 740 * 64
0.486486486486
Mean acc: 0.5140540540540541


In [54]:
Bd[0][-1]

array([0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,

In [ ]:
mio.export_pickle(Bd, '/homes/yz4009/wd/PickleModel/EarRecognition/verification_bench_align_PEP.pkl', overwrite=False)

#### WPUTEDB

In [ ]:
protocol_wputedb = mio.import_pickle('{}/{}/protocol.pkl'.format(db_path,'WPUTEDB'), encoding='latin1')

In [ ]:
Ad = cross_validation(protocalAndPath(protocol_wputedb, db_path/'WPUTEDB'/db_type,ext), clf, postfix=feature, preprocess=preprocess)
print('Mean acc: {}'.format(np.mean(list(zip(*Ad))[0])))

In [ ]:
Ad[0][-1]

#### Cross Test

In [ ]:
civ_result = cross_intersect_validation(protocalAndPath(protocol_wputedb, db_path/'WPUTEDB'/db_type,ext),protocalAndPath(protocol_vggear, db_path/'VGGEAR'/db_type,ext), clf, postfix=feature, preprocess=preprocess)

In [ ]:
Ac,Bc = list(zip(*civ_result))
print('Mean acc Ac: {}'.format(np.mean(list(zip(*Ac))[0])))
print('Mean acc Bc: {}'.format(np.mean(list(zip(*Bc))[0])))

In [ ]:
adv = np.mean(list(zip(*Ad))[0])
bdv = np.mean(list(zip(*Bd))[0])
acv = np.mean(list(zip(*Ac))[0])
bcv = np.mean(list(zip(*Bc))[0])

print([adv,acv,bdv,bcv,bcv-bdv,adv-acv])

#### store results

In [ ]:
mio.export_pickle([Ad,Ac,Bd,Bc], '/homes/yz4009/wd/PickleModel/EarRecognition/verification_unaligned_DSIFT_SVM.pkl', overwrite=False)

## Visualise Results

In [ ]:
[Ad,Ac,Bd,Bc] = mio.import_pickle('/homes/yz4009/wd/PickleModel/EarRecognition/verification_unaligned_DSIFT_SVM.pkl')

In [2]:
from menpo.visualize import plot_curve
def roc_plot(methods_results, legend_names="", bbox_to_anchor=(1.3, 0.5), figsize=(5,7)):
    plt.figure(figsize=figsize)

    num_folds = len(methods_results[0])

    x_axis = []
    y_axis = []
    for method_id,results in enumerate(methods_results):

        mean_tpr = 0.0
        mean_fpr = np.linspace(0, 1, 100)

        for i in range(num_folds):
            mean_tpr += np.interp(mean_fpr, results[i][1], results[i][2])
            mean_tpr[0] = 0.0

        mean_tpr /= num_folds
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)

        plt.plot(mean_fpr, mean_tpr, '--', lw=3)

        x_axis.append(mean_fpr)
        y_axis.append(mean_tpr)

        print('{:} Auc: {:5.3f}, Mean: {:6.4f}, Std: {:5.3f}'.format(legend_names[method_id], mean_auc, np.mean([r[0] for r in results]), np.std([r[0] for r in results])))
        
#     plot_curve(np.array(x_axis).T,np.array(y_axis).T)

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.ylabel('True positive rate (TPR)')
    plt.xlabel('False positive Rate (FPR)')
    plt.title('ROC curve')
    plt.tick_params(labelsize=14)

    plt.legend(legend_names, loc=10, bbox_to_anchor=bbox_to_anchor)
    plt.show()

In [ ]:
roc_plot([Ad, Ac, Bd, Bc], ['WPUTEDB+Direct+PI', 'WPUTEDB+Cross+PI', 'Our DB+Direct+PI', 'Our DB+Cross+PI'])

## Report Verification Comparision

In [ ]:
results_compare_aligned = []
results_compare_aligned_labels = []
for method in ['LDA', 'SVM']:
    for feature in ['PI','DSIFT']:
        results_compare_aligned.append(
            mio.import_pickle('/homes/yz4009/wd/PickleModel/EarRecognition/verification_unaligned_{}_{}.pkl'.format(feature, method))
        )
        results_compare_aligned_labels.append([
            'WPUTEDB+Direct+{}+{}'.format(feature, method), 
            'WPUTEDB+Cross+{}+{}'.format(feature, method), 
            'Our DB+Direct+{}+{}'.format(feature, method), 
            'Our DB+Cross+{}+{}'.format(feature, method)
        ])
        
results_compare_aligned = np.concatenate(results_compare_aligned)
results_compare_aligned_labels = np.concatenate(results_compare_aligned_labels)

In [ ]:
roc_plot(results_compare_aligned, results_compare_aligned_labels, bbox_to_anchor=(1.2, 0.4))

In [ ]:
results = {
    'SVM+int': [0.62702702702702706, 0.54864864864864871, 0.52702702702702697, 0.59999999999999998, 0.072972972972973005, 0.078378378378378355],
    'LDA+int': [0.64648648648648654, 0.54918918918918913, 0.56594594594594594, 0.57081081081081086, 0.0048648648648649262, 0.097297297297297414],
#     'SVM+igo':[0.62046649888935368, 0.58349243135459317, 0.6123870687392875, 0.52932042172716209, -0.083066647012125405, 0.036974067534760513],
#     'LDA+igo':[0.62667534650736612, 0.56914678977656408, 0.6176923032157684, 0.5676226300631031, -0.050069673152665306, 0.057528556730802038],
    'LDA+int+align':[0.67675675675675673, 0.57729729729729728, 0.56054054054054059, 0.65621621621621617, 0.095675675675675587, 0.099459459459459443],
    'SVM+int+align':[0.65729729729729724, 0.57243243243243236, 0.55675675675675684, 0.64756756756756739, 0.090810810810810549, 0.084864864864864886],
#     'SVM+dsift+align':[0.50148215499417503, 0.49853543197716998, 0.50658903452281734, 0.51758557246870907, 0.010996537945891727, 0.0029467230170050485],
#     'LDA+dsift+align':[0.68480749310169531, 0.58961469083626228, 0.65152646116141866, 0.55939885707195658, -0.092127604089462078, 0.09519280226543303],
#     'LDA+dsift': [0.7005405405405406, 0.58918918918918917, 0.62594594594594599, 0.54162162162162164, -0.084324324324324351, 0.11135135135135144],
    'LDA+PEP+PCA+align': [0.64540540540540536, 0.58648648648648638, 0.61891891891891893, 0.63729729729729734, 0.018378378378378413, 0.058918918918918983],
    'SVM+PEP+PCA+align': [0.62378378378378374, 0.55459459459459448, 0.60918918918918918, 0.58756756756756756, -0.021621621621621623, 0.06918918918918926],
#     'SVM+dsiff': [0.5077264120799666, 0.50158446895955011, 0.49502824983447358, 0.49941304347826082, 0.0043847936437872437, 0.0061419431204164932],
    'LDA+PEP+align': [0.65405405405405403, 0.58864864864864852, 0.62216216216216214, 0.63891891891891883, 0.016756756756756697, 0.065405405405405515],
    'SVM+PEP+align':[0.63891891891891883, 0.5637837837837838, 0.60324324324324319, 0.58108108108108103, -0.022162162162162158, 0.075135135135135034],
}

In [ ]:
key_len = 18
print(' method             &   Ad  &   Ac  &   Bd  &   Bc  &  Bc-Bd &  Ad-Ac \\\\')
for k in results.keys():
    res = results[k]
#     print(' {} & {:4.1f}% & {:4.1f}% & {:4.1f}% & {:4.1f}% & {:5.1f}% & {:5.1f}% \\\\'.format(k + ' '*(key_len-len(k)),res[0]*100.0,res[1]*100.0,res[2]*100.0,res[3]*100.0,res[4]*100.0,res[5]*100.0))
    print(' {} & {:6.4f} & {:6.4f} & {:6.4f} & {:6.4f} & {:7.4f} & {:7.4f} \\\\'.format(k + ' '*(key_len-len(k)),res[0],res[1],res[2],res[3],res[4],res[5]))

## Report Verification Benchmark

In [3]:
alignment = 'align'
results_compare_aligned = []
results_compare_aligned_labels = []
for method in ['PI','DSIFT', 'PEP', 'FV']:
    results_compare_aligned.append([
        mio.import_pickle('/homes/yz4009/wd/PickleModel/EarRecognition/verification_bench_{}_{}.pkl'.format(alignment,method))
    ])
    results_compare_aligned_labels.append([
        '{}'.format(method),
    ])
        
results_compare_aligned = np.concatenate(results_compare_aligned)
results_compare_aligned_labels = np.concatenate(results_compare_aligned_labels)

AttributeError: Can't get attribute 'JBC' on <module '__main__'>

In [4]:
roc_plot(results_compare_aligned, results_compare_aligned_labels, bbox_to_anchor=(1.2, 0.4))

ValueError: setting an array element with a sequence.